In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from StereoNet_single import StereoNet
from StereoNet_single import StereoNetOnlyRefine
import open3d as o3d
import matplotlib.pyplot as plt
from kinfu_cv import KinfuPlarr

In [ ]:
focal_length = 643.338013 #942.8       # lense focal length, 1.88mm, 942.8 ???  643.338013, 643.096008
baseline = 55   #49.75  distance in mm between the two cameras
units = 0.512     # depth units, adjusted for the output to fit in one byte

width = 640
height = 480

# intrinsicsNp = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

## FocalLengthLeft: 643.338379 643.096497
## PrincipalPointLeft: 638.957336 402.329956
## PinholeCameraIntrinsic(width: int, height: int, fx: float, fy: float, cx: float, cy: float)
pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, 
                                                             643.338013, 643.096008, 
                                                             638.956970, 402.330017);

print('Camera intrinsics:')
print(pinhole_camera_intrinsic.intrinsic_matrix)

In [ ]:
def convertDisparityMapToDepthMap(disparityMap):
    # shape: disparityMap.shape
    valid_pixels = disparityMap > 0
    depth = np.zeros(shape=disparityMap.shape).astype("uint16")
    depth[valid_pixels] = (focal_length * baseline) / (units * disparityMap[valid_pixels])
    
    return depth

def convertDepthMapToDisparityMap(depthMap):
    # shape: depthMap.shape
    valid_pixels = depthMap > 0
    disparity = np.zeros(shape=depthMap.shape).astype("float32")
    disparity[valid_pixels] = (focal_length * baseline) / (units * depthMap[valid_pixels])
    
    return disparity

In [ ]:
checkpoint = torch.load('./data/checkpoint_pretrain_secneflow.pth', map_location=torch.device('cpu'))

stereoNetModel = StereoNet(k=4-1, r=4-1, maxdisp=192)
stereoNetModel = nn.DataParallel(stereoNetModel)
stereoNetModel.load_state_dict(checkpoint['state_dict'])
stereoNetModel.eval();

stereoNetOnlyRefineModel = StereoNetOnlyRefine(k=4-1, r=4-1, maxdisp=192)
stereoNetOnlyRefineModel = nn.DataParallel(stereoNetOnlyRefineModel)
stereoNetOnlyRefineModel.load_state_dict(checkpoint['state_dict'])
stereoNetOnlyRefineModel.eval();

def convertImgToModelInput2(img):
    normalize = {'mean': [0.0, 0.0, 0.0], 'std': [1.0, 1.0, 1.0]}
    
    imgTmp = img
    imgTmp = imgTmp.astype(float)
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(**normalize)
    ])
    
    imgTmp = transform(imgTmp).float()
    return imgTmp

def convertImgToModelInput(img):
    normalize = {'mean': [0.0, 0.0, 0.0], 'std': [1.0, 1.0, 1.0]}
    
    imgTmp = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    imgTmp = imgTmp.astype(float)
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(**normalize)
    ])
    
    imgTmp = transform(imgTmp).float()
    return imgTmp

def getStereoNet(left, right, useRefinedDisparity = True):
#     normalize = {'mean': [0.0, 0.0, 0.0], 'std': [1.0, 1.0, 1.0]}
#     m = left.shape[0] # 480
#     n = right.shape[1] # 640

    imgL = convertImgToModelInput(left)
    imgR = convertImgToModelInput(right)
    
#     imgL = cv2.cvtColor(left, cv2.COLOR_GRAY2RGB)
#     imgR = cv2.cvtColor(right, cv2.COLOR_GRAY2RGB)
    
#     imgL = imgL.astype(float)
#     imgR = imgR.astype(float)

#     transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(**normalize)
#     ])

#     imgL = transform(imgL).float()
#     imgR = transform(imgR).float()

    outputs = None
    with torch.no_grad():
        imgL1 = imgL.unsqueeze(0)
        imgR1 = imgR.unsqueeze(0)
        outputs = stereoNetModel(imgL1, imgR1)

#         print('model output')
#         print(len(outputs))
#         print(outputs[0].shape)
#         print(outputs[1].shape)

    sn_disparity = outputs[0].squeeze(0).cpu().numpy()
    sn_disparity_refined = outputs[1].squeeze(0).cpu().numpy()
    
    disparity_to_use = sn_disparity
    if useRefinedDisparity:
        #disparity_to_use = disparity_to_use + sn_disparity_refined
        disparity_to_use = sn_disparity_refined
    
    # convert to depth map
    sn_depth_map = convertDisparityMapToDepthMap(disparity_to_use)
    sn_depth_map[disparity_to_use < 0] = 0

    return sn_depth_map, sn_disparity

def getRefinedDepthMap(depth, left):
    rs_disparityMap = convertDepthMapToDisparityMap(depth) # convert depth to disparity
    rs_disparityMap_tensor = torch.tensor(rs_disparityMap).float()
    
    imgL = convertImgToModelInput2(left)    
    output1 = None
    with torch.no_grad():
        imgL1 = imgL.unsqueeze(0)
        rs_disparityMap_tensor = rs_disparityMap_tensor.unsqueeze(0)
        output1 = stereoNetOnlyRefineModel(rs_disparityMap_tensor, imgL1)
        
    refined_rs_disparity = output1.squeeze(0).cpu().numpy()
    refined_rs_depth = convertDisparityMapToDepthMap(refined_rs_disparity)
    
    return refined_rs_depth, refined_rs_disparity

In [ ]:
def getPointCloud(rgb_img, depth_image):
    # expects color image to be 8-bits per pixel
    # expects depth image to be 16-bits per pixel
    
    o3dColorImage1 = o3d.geometry.Image(rgb_img)
    o3dDepthImage1 = o3d.geometry.Image(depth_image)

    # Create an RGBD open3d Image
    frame_rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(o3dColorImage1, o3dDepthImage1, 
                                                                          convert_rgb_to_intensity=False)

    # Create Point cloud
    framePC = o3d.geometry.PointCloud.create_from_rgbd_image(frame_rgbd_image, pinhole_camera_intrinsic)
    
    
#   framePC = o3d.geometry.PointCloud.create_from_depth_image(o3dDepthImage1, pinhole_camera_intrinsic, 
#                                                                   depth_scale=1000.0)
    
    return framePC

In [ ]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

res_x = width
res_y = height

config.enable_stream(rs.stream.depth, res_x, res_y, rs.format.z16, 30)
config.enable_stream(rs.stream.color, res_x, res_y, rs.format.bgr8, 30)

config.enable_stream(rs.stream.infrared, 1, res_x, res_y, rs.format.y8, 30)
config.enable_stream(rs.stream.infrared, 2, res_x, res_y, rs.format.y8, 30)

# Start streaming
pipeline_profile = pipeline.start(config)

## To set laser on/off or set laser power
device = pipeline_profile.get_device()
depth_sensor = device.query_sensors()[0]
laser_pwr = depth_sensor.get_option(rs.option.laser_power)
print("laser power = ", laser_pwr)
laser_range = depth_sensor.get_option_range(rs.option.laser_power)
print("laser power range = " , laser_range.min , "~", laser_range.max)
depth_sensor.set_option(rs.option.laser_power, 0)

f1 = pipeline.wait_for_frames()
d1 = f1.get_depth_frame()


#// Depth scale is needed for the kinfu_plarr set-up
depth_sensor = pipeline_profile.get_device().first_depth_sensor()

# depth_sensor.set_option(rs.option.visual_preset, rs.option.RS400_VISUAL_PRESET_HIGH_DENSITY);
preset_range = depth_sensor.get_option_range(rs.option.visual_preset)
print('preset range:'+str(preset_range))

for i in range(int(preset_range.max)):
    visulpreset = depth_sensor.get_option_value_description(rs.option.visual_preset,i)
    print(i, visulpreset)
    
    if visulpreset == "High Accuracy":
        depth_sensor.set_option(rs.option.visual_preset, i)

depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)


spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 3)
spatial.set_option(rs.option.filter_smooth_alpha, 0.5)
spatial.set_option(rs.option.filter_smooth_delta, 20)
spatial.set_option(rs.option.holes_fill, 0)
temporal = rs.temporal_filter()
# decimation = rs.decimation_filter()
# decimation.set_option(rs.option.filter_magnitude, 4)
hole_filling = rs.hole_filling_filter()
hole_filling.set_option(rs.option.holes_fill, 1)

# Image directory 
directory = r'./outputs/'
img_count = 1

kfp = KinfuPlarr(width, height, depth_scale, 643.338013, 643.096008, 638.95697, 402.330017, True)

rayCastedDepth = np.zeros(shape=(height, width))

cc = 0
try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        ir1_frame = frames.get_infrared_frame(1) # Left IR Camera, it allows 0, 1 or no input
        ir2_frame = frames.get_infrared_frame(2) # Right IR camera
        if not depth_frame or not color_frame:
            continue

        if not ir1_frame or not ir2_frame:
            continue
            
        # skipping first 20 frames, to wait for sensor for warm up
        cc = cc + 1
        if cc < 20:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data()) # dtype of print(depth_image.dtype) is: uint16
        color_image = np.asanyarray(color_frame.get_data()) # FORMAT: BGR, dtype of print(color_image.dtype) is: uint8
        color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        
        ir1_image = np.asanyarray(ir1_frame.get_data()) # Left image
        ir2_image = np.asanyarray(ir2_frame.get_data()) # Right image
        
        left = ir1_image
        right = ir2_image
        rs_depth = depth_image # (image of 16-bit per pixel)
        
        ## Apply filters to real-sense depth
        filtered_rs_depth = spatial.process(depth_frame)
        filtered_rs_depth = temporal.process(filtered_rs_depth)
        filtered_rs_depth = hole_filling.process(filtered_rs_depth)
        filtered_rs_depth = np.asanyarray(filtered_rs_depth.get_data()) # dtype of print(filtered_rs_depth.dtype) is: uint16
        
#         ## Apply bilateral filter to RS depth image BEFORE RS filters
#         tmp = rs_depth.astype(np.float32)
#         tmp = cv2.bilateralFilter(tmp, 9, 75, 75)
#         bilateral_depth = tmp.astype(np.uint16)
        
#         ## Apply bilateral filter to RS depth image AFTER RS filters
#         tmp = filtered_rs_depth.astype(np.float32)
#         tmp = cv2.bilateralFilter(tmp, 9, 75, 75)
#         filtered_bilateral_depth = tmp.astype(np.uint16)
        
        
#         print('# of valid depth values [rs_depth]: ', len(rs_depth[rs_depth > 0]))
#         print(rs_depth[rs_depth > 0])
#         print('# of valid depth values [bilateral_depth]: ', len(bilateral_depth[bilateral_depth > 0]))
#         print(bilateral_depth[bilateral_depth > 0])
        
#         print('# of valid depth values [filtered_rs_depth]: ', len(filtered_rs_depth[filtered_rs_depth > 0]))
#         print(filtered_rs_depth[filtered_rs_depth > 0])
#         print('# of valid depth values [filtered_bilateral_depth]: ', len(filtered_bilateral_depth[filtered_bilateral_depth > 0]))
#         print(filtered_bilateral_depth[filtered_bilateral_depth > 0])
        
    
    
    
        ## Use StereoNet to estimate depth
        sn_depth_map, sn_disparity = getStereoNet(left, right, False)
        
    
#         print('StereoNet Disparitymap')
#         print(sn_disparity.shape, sn_disparity.dtype)
        
#         print('StereoNet Depthmap')
#         print(sn_depth_map.shape, sn_depth_map.dtype)
        
#         print('# of valid depth values [sn_depth_map]: ', len(sn_depth_map[sn_depth_map > 0]))
#         print(sn_depth_map[sn_depth_map > 0])


        ## Use Edge Refinement Network to refine depth from RealSense
        #refined_rs_depth, refined_rs_disparity = getRefinedDepthMap(bilateral_depth, color_image_rgb)
        
        ## Kinect Fusion
#         depth_image_flatten = np.array(depth_image).flatten()
#         depth_image_flatten = depth_image_flatten.tolist()
#         isSuccess = kfp.integrateFrame(depth_image_flatten)
#         if isSuccess:
#             pose = kfp.getPose()
#             pose = np.asarray(pose, dtype=np.float32, order='C').reshape((4, -1))
# #             print(pose.shape)
# #             print(pose.dtype)
#             print(pose)
            
#             d = kfp.getCurrentDepth()
#             d = np.array(d)
#             d = np.nan_to_num(d)
#             d = d.reshape((height, -1))
#             d = d * (1/depth_scale)
#             d = d.astype(np.int16)
            
#             rayCastedDepth = d
            
#             kfp.renderShow()
                    
            
            
            

        #### VISUALISATION ####
        is_visual_on = True
        if not is_visual_on:
            continue
        
        ## Visualize RGB frame
#         cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
#         cv2.imshow('RealSense', color_image)
        
        ## Visualize IR frames
        ir_images = np.hstack((ir1_image, ir2_image))
        cv2.namedWindow('IRSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('IRSense', ir_images)
        
        
        
#         ## Visualize Real sense depth frames
#         ## Apply colormap on Real-sense depth image 
#         rs_depth_sccaled  = cv2.convertScaleAbs(depth_image, alpha=0.03)  # (image converted to 8-bit per pixel)
#         filtered_rs_depth_scaled = cv2.convertScaleAbs(filtered_rs_depth, alpha=0.03)  # (image converted to 8-bit per pixel)
#         #depth_colormap = cv2.applyColorMap(rs_depth_sccaled, cv2.COLORMAP_JET)
#         depth_colormap = cv2.applyColorMap(cv2.equalizeHist(rs_depth_sccaled), cv2.COLORMAP_JET)
#         #filtered_depth_colormap = cv2.applyColorMap(filtered_rs_depth_scaled, cv2.COLORMAP_JET)
#         filtered_depth_colormap = cv2.applyColorMap(cv2.equalizeHist(filtered_rs_depth_scaled), cv2.COLORMAP_JET)
#         both_depths2 = np.hstack((depth_colormap, filtered_depth_colormap))
#         cv2.namedWindow('RS_DepthMap', cv2.WINDOW_AUTOSIZE)
#         cv2.imshow('RS_DepthMap', both_depths2)
        
        
        
        
        ## Visualize Real sense depth frames with Bilateral filter
#         bilateral_depth_colormap = cv2.applyColorMap(cv2.equalizeHist(cv2.convertScaleAbs(bilateral_depth, alpha=0.03)), cv2.COLORMAP_JET)
#         bilateral_filtered_depth_colormap = cv2.applyColorMap(cv2.equalizeHist(cv2.convertScaleAbs(filtered_bilateral_depth, alpha=0.03)), cv2.COLORMAP_JET)
#         both_depths2 = np.hstack((bilateral_depth_colormap, bilateral_filtered_depth_colormap))
#         cv2.namedWindow('RS_Bilateral_DepthMap', cv2.WINDOW_AUTOSIZE)
#         cv2.imshow('RS_Bilateral_DepthMap', both_depths2)
        
        
        
        
# #         ## Visualize rayCastedDepth
#         tmp1 = cv2.convertScaleAbs(rayCastedDepth, alpha=0.03)
#         rayCastedDepth_colormap = cv2.applyColorMap(cv2.equalizeHist(tmp1), cv2.COLORMAP_JET)
#         cv2.namedWindow('rayCastedDepth', cv2.WINDOW_AUTOSIZE)
#         cv2.imshow('rayCastedDepth', rayCastedDepth_colormap)
        
        
        ## Visualize stereoNet disparity and depth maps (StereoNET)
        tmp1 = cv2.convertScaleAbs(sn_disparity.astype(np.uint16))
        o1 = cv2.applyColorMap(cv2.equalizeHist(tmp1), cv2.COLORMAP_JET)
        o1[sn_disparity < 0] = 0
        o2 = cv2.applyColorMap(cv2.equalizeHist(cv2.convertScaleAbs(sn_depth_map, alpha=0.03)), cv2.COLORMAP_JET)
        o2[sn_depth_map < 0] = 0
        #stn_images = o1
        stn_images = np.hstack((o1, o2))
        cv2.namedWindow('StereoNet', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('StereoNet', stn_images)
        
        

        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        if key == 116:
#             cv2.imwrite(directory + str(img_count) + '_color_image.jpg', color_image)
#             cv2.imwrite(directory + str(img_count) + '_left.jpg', left)
#             cv2.imwrite(directory + str(img_count) + '_right.jpg', right)
#             cv2.imwrite(directory + str(img_count) + '_rs_depth.jpg', depth_colormap)
#             cv2.imwrite(directory + str(img_count) + '_rs_filtered_depth.jpg', filtered_depth_colormap)
#             cv2.imwrite(directory + str(img_count) + '_bm_depth.jpg', temp1)
#             cv2.imwrite(directory + str(img_count) + '_bm_filtered_depth.jpg', temp2)
#             cv2.imwrite(directory + str(img_count) + '_bm_disparity.jpg', raw_disparity_scaled)
#             cv2.imwrite(directory + str(img_count) + '_bm_filtered_disparity.jpg', filtered_disparity_scaled)
            img_count = img_count+1
        if key == ord('1'):
            rs_pc = getPointCloud(color_image_rgb, rs_depth)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
        if key == ord('2'):
            rs_pc = getPointCloud(color_image_rgb, filtered_rs_depth)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
#         if key == ord('3'):
#             rs_pc = getPointCloud(color_image_rgb, bilateral_depth)
#             rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
#             print('rs_pc', rs_pc)
#             o3d.visualization.draw_geometries([rs_pc])
        if key == ord('4'):
            rs_pc = getPointCloud(color_image_rgb, sn_depth_map)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
        if key == ord('5'):
            rs_pc = getPointCloud(color_image_rgb, rayCastedDepth)
            rs_pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
            print('rs_pc', rs_pc)
            o3d.visualization.draw_geometries([rs_pc])
        if key == ord('t'):
            cv2.imwrite(directory + str(img_count) + '_color_image.jpg', color_image)
            cv2.imwrite(directory + str(img_count) + '_left.jpg', left)
            cv2.imwrite(directory + str(img_count) + '_right.jpg', right)
            cv2.imwrite(directory + str(img_count) + '_rs_depth.jpg', depth_colormap)
            cv2.imwrite(directory + str(img_count) + '_rs_filtered_depth.jpg', filtered_depth_colormap)
            cv2.imwrite(directory + str(img_count) + '_raycasted_depth.jpg', rayCastedDepth_colormap)
            img_count = img_count+1
        if key == ord('y'):
            cv2.imwrite(directory + str(img_count) + '_color_image.jpg', color_image)
            cv2.imwrite(directory + str(img_count) + '_left.jpg', left)
            cv2.imwrite(directory + str(img_count) + '_right.jpg', right)
            cv2.imwrite(directory + str(img_count) + '_stereonet_depth.jpg', o2)
            img_count = img_count+1
            
finally:

    # Stop streaming
    pipeline.stop()